In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

In [2]:
# Load and preprocess the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values to [0, 1]
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

In [3]:
# Create a base model (MobileNetV2) with pretrained weights (excluding the top)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(32, 32, 3)))

9406464/9406464 [==============================] - 1s 0us/step


In [4]:
# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

In [5]:
# Add custom layers on top of the base model
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(10, activation='softmax')(x)

In [6]:
# Create the final transfer learning model
transfer_model = Model(inputs=base_model.input, outputs=output)

In [7]:
# Compile the model
transfer_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Print a summary of the model architecture
transfer_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 16, 16, 32)           864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 16, 16, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 16, 16, 32)           0         ['bn_Conv1[0][0]']        

In [9]:
# Train the transfer learning model
history = transfer_model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/10
1563/1563 [==============================] - 63s 37ms/step - loss: 1.9307 - accuracy: 0.3009 - val_loss: 1.8638 - val_accuracy: 0.3276
Epoch 2/10
1563/1563 [==============================] - 35s 23ms/step - loss: 1.8316 - accuracy: 0.3347 - val_loss: 1.8311 - val_accuracy: 0.3369
Epoch 3/10
1563/1563 [==============================] - 36s 23ms/step - loss: 1.7932 - accuracy: 0.3484 - val_loss: 1.8123 - val_accuracy: 0.3449
Epoch 4/10
1563/1563 [==============================] - 36s 23ms/step - loss: 1.7646 - accuracy: 0.3597 - val_loss: 1.8028 - val_accuracy: 0.3483
Epoch 5/10
1563/1563 [==============================] - 36s 23ms/step - loss: 1.7424 - accuracy: 0.3675 - val_loss: 1.7955 - val_accuracy: 0.3485
Epoch 6/10
1563/1563 [==============================] - 35s 22ms/step - loss: 1.7233 - accuracy: 0.3746 - val_loss: 1.7920 - val_accuracy: 0.3569
Epoch 7/10
1563/1563 [==============================] - 35s 22ms/step - loss: 1.7065 - accuracy: 0.3800 - val_loss: 1.7901 -

In [10]:
# Evaluate the model on the test dataset
test_loss, test_acc = transfer_model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

313/313 [==============================] - 6s 19ms/step - loss: 1.8004 - accuracy: 0.3527
Test accuracy: 0.35269999504089355
